## Plan
- [ ] follow https://duckdb.org/docs/guides/python/jupyter.html
- [ ] follow https://github.com/onefact/datathinking.org-codespace/blob/main/notebooks/in-class-notebooks/230406-data-build-tool-for-new-york-city-311-phone-call-data.ipynb
- [ ] load csv file in duckdb

In [1]:
import duckdb
import pandas as pd
# No need to import duckdb_engne
#  jupysql will auto-detect thei driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [2]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False


In [4]:
%sql duckdb:///:memory:

In [7]:
!wget https://data.payless.health/health.data.ny.gov%2FNYSDOH_BRFSS_SurveyData_2021.csv -O /tmp/NYSDOH_BRFSS_SurveyData_2021.csv

--2023-04-10 16:05:47--  https://data.payless.health/health.data.ny.gov%2FNYSDOH_BRFSS_SurveyData_2021.csv
Resolving data.payless.health (data.payless.health)... 18.164.116.15, 18.164.116.105, 18.164.116.95, ...
Connecting to data.payless.health (data.payless.health)|18.164.116.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407489625 (389M) [text/csv]
Saving to: '/tmp/NYSDOH_BRFSS_SurveyData_2021.csv'

/tmp/NYSDOH_BRFSS_S 100%[===================>] 388.61M  43.7MB/s    in 9.8s    

2023-04-10 16:05:57 (39.9 MB/s) - '/tmp/NYSDOH_BRFSS_SurveyData_2021.csv' saved [407489625/407489625]



## Documentation to load csv to duckdb for use in SQL queries
https://duckdb.org/docs/data/csv/overview.html

In [6]:
%%sql

SELECT * FROM read_csv_auto('/tmp/NYSDOH_BRFSS_SurveyData_2021.csv') LIMIT 10

(duckdb.InvalidInputException) Invalid Input Error: Error in file "/tmp/NYSDOH_BRFSS_SurveyData_2021.csv" at line 2 in column "17": Invalid unicode (byte sequence mismatch) detected in CSV file. Parser options:
  file=/tmp/NYSDOH_BRFSS_SurveyData_2021.csv
  delimiter=',' (auto detected)
  quote='"' (auto detected)
  escape='' (auto detected)
  header=0 (auto detected)
  sample_size=20480
  ignore_errors=0
  all_varchar=0
[SQL: SELECT * FROM read_csv_auto('/tmp/NYSDOH_BRFSS_SurveyData_2021.csv') LIMIT 10]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [8]:
%%sql

SELECT * FROM '/tmp/NYSDOH_BRFSS_SurveyData_2021.csv' LIMIT 10

(duckdb.InvalidInputException) Invalid Input Error: Error in file "/tmp/NYSDOH_BRFSS_SurveyData_2021.csv" at line 2 in column "17": Invalid unicode (byte sequence mismatch) detected in CSV file. Parser options:
  file=/tmp/NYSDOH_BRFSS_SurveyData_2021.csv
  delimiter=',' (auto detected)
  quote='"' (auto detected)
  escape='' (auto detected)
  header=0 (auto detected)
  sample_size=20480
  ignore_errors=0
  all_varchar=0
[SQL: SELECT * FROM '/tmp/NYSDOH_BRFSS_SurveyData_2021.csv' LIMIT 10]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [ ]:
%%sql

SELECT * FROM '/tmp/NYSDOH_BRFSS_SurveyData_2021.csv' LIMIT 10

In [9]:
%%sql
-- read a CSV from stdin, auto-infer options
cat data/csv/issue2471.csv | duckdb -c "select * from read_csv_auto('/dev/stdin')"

(duckdb.ParserException) Parser Error: syntax error at or near "cat"
LINE 2: cat data/csv/issue2471.csv | duckdb -c "select * from read_csv_auto('/dev/stdin')"...
        ^
[SQL: -- read a CSV from stdin, auto-infer options
cat data/csv/issue2471.csv | duckdb -c "select * from read_csv_auto('/dev/stdin')"]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [11]:
!cat data/csv/issue2471.csv | duckdb -c "select * from read_csv_auto('/dev/stdin')"

cat: data/csv/issue2471.csv: No such file or directory
zsh:1: command not found: duckdb
